In [1]:
from model.UNet import UNet
from model.DeepLabV3Plus import DeepLabV3Plus
from model.HRNetV2 import HRNetV2

import os, gc

from tqdm import tqdm
from IPython.display import clear_output

import numpy as np
import torch
import torch.nn as nn
from torchvision import transforms
from torchvision.transforms import InterpolationMode
import torchvision.transforms.functional as TF
from torch.utils.data import Dataset, DataLoader

import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(device)

cuda


In [3]:
import cv2

import random

class SegmentationDataset(Dataset):
  def __init__(self, data_type, mode):    
    self.mode = mode
    self.input, self.label = [], []

    # 작업할 폴더의 경로
    path = './dataset/{0}s'.format(data_type) 

    self.input_path = '{0}/{1}/{2}'.format(path, self.mode, "data")
    self.label_path = '{0}/{1}/{2}'.format(path, self.mode, "label")

    # 폴더 안에 있는 데이터 파일들의 이름을 추출
    data_names = [name.split('.')[0] for name in os.listdir(self.input_path)]

    # 데이터 전처리
    for data_name in data_names:
        input = '{0}/{1}.png'.format(self.input_path, data_name)
        label = '{0}/{1}.png'.format(self.label_path, data_name)

        #""" 컴퓨터 메모리(RAM)가 부족한 경우, 아래를 주석 해제하고 이 부분을 주석 처리
        input = cv2.cvtColor(cv2.imread(input), cv2.COLOR_BGR2RGB)
        label = cv2.imread(label, cv2.IMREAD_GRAYSCALE)

        input = torch.FloatTensor(input).permute(2, 0, 1)
        label = torch.LongTensor(label)
        #"""

        self.input.append(input)
        self.label.append(label)

  def __len__(self):
    return len(self.input)

  def __getitem__(self, index):
    input = self.input[index]
    label = self.label[index]
    
    """ 컴퓨터 메모리(RAM)가 부족한 경우, 위를 주석 처리하고 이 부분을 주석 해제
    input_img = cv2.cvtColor(cv2.imread(input), cv2.COLOR_BGR2RGB)
    label_img = cv2.imread(label, cv2.IMREAD_GRAYSCALE)

    input = torch.FloatTensor(input_img).permute(2, 0, 1)
    label = torch.LongTensor(label_img)
    """

    label = label.unsqueeze(dim=0) 
    
    params = transforms.RandomResizedCrop.get_params(input, scale=(0.5, 0.5), ratio=(3.0 / 4.0, 4.0 / 3.0))

    input = TF.resized_crop(input, *params, size=(512, 512), interpolation=InterpolationMode.BILINEAR)
    label = TF.resized_crop(label, *params, size=(512, 512), interpolation=InterpolationMode.NEAREST)

    label = label.squeeze(dim=0)

    # 좌우 뒤집기
    if random.random() > 0.5:
        input = TF.hflip(input)
        label = TF.hflip(label)

    # 상하 뒤집기
    if random.random() > 0.5:
        input = TF.vflip(input)
        label = TF.vflip(label)

    # 무작위 회전 각도 얻기
    #angle = random.uniform(-60, 60)

    # 이미지 및 라벨 무작위 회전
    #input = TF.affine(input, angle=angle, translate=(0, 0), scale=1.0, shear=(0, 0), interpolation=InterpolationMode.BILINEAR)
    #label = TF.affine(label, angle=angle, translate=(0, 0), scale=1.0, shear=(0, 0), interpolation=InterpolationMode.NEAREST)

    return input, label

In [4]:
train_dataset   = SegmentationDataset('building', 'train')
test_dataset    = SegmentationDataset('building', 'test')

#train_dataset   = SegmentationDataset('building', 'train')
#test_dataset    = SegmentationDataset('building', 'test')

In [5]:
from sklearn.metrics import accuracy_score, f1_score

def evaluate(model, dataloader):
  
  model.eval()

  predvs, labels = [], []

  with torch.no_grad():
    for batch in tqdm(dataloader, desc="Test Processed"):
        input, label = batch

        input = input.to(device)

        hypothesis = model(input)

        predv = torch.argmax(hypothesis, dim=1).cpu()

        predvs.extend(predv.numpy().ravel())
        labels.extend(label.numpy().ravel())

    # 평가 지표 mIoU를 추가할 것
    print("Accuracy - {0:f}\n".format(accuracy_score(labels, predvs)))
    print("F1 Score - {0:f}\n".format(f1_score(labels, predvs)))

In [6]:
trained_model_path = "./trained"

def train(model : nn.Module, dataset, num_epoch = 10, lr=0.001):
    if not hasattr(model, "current_epoch") is None: model.current_epoch = 0

    dataloader = DataLoader(dataset, batch_size=8, shuffle=True) 

    loss_func = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    for epoch in range(model.current_epoch, model.current_epoch + num_epoch):
        print("# | Epoch - {0:03d} / {1:03d} | #".format(epoch + 1, model.current_epoch + num_epoch))
        print("=========================")
        
        model.train()
        
        costs = []

        for batch in tqdm(dataloader, desc="Batch Processed"):
            input, label = batch

            input = input.to(device)
            label = label.to(device)

            optimizer.zero_grad()

            hypothesis = model(input)
            
            cost = loss_func(hypothesis, label)

            cost.backward()
            optimizer.step()

            costs.append(cost.item())

        print("Current Average Loss - {0:f}".format(np.mean(costs)))

        if (epoch + 1) % 10 == 0:
            path = "{0}/{1}/epoch_{2:05d}.pt".format(trained_model_path, model.__class__.__name__, epoch + 1)
            
            torch.save(model.state_dict(), path)

            print("모델 파일({0}) 저장됨\n".format(path))
        
        #evaluate(model, dataloader)
    
    torch.cuda.empty_cache()
    gc.collect()

In [7]:
def load_trained_model(model : nn.Module, epoch):
    model.load_state_dict(torch.load('{0}/{1}/epoch_{2:05d}.pt'.format(trained_model_path, model.__class__.__name__, epoch)))

    model.current_epoch = epoch

In [8]:
num_epoch = 100

torch.cuda.empty_cache()
gc.collect()

#model1 = UNet().to(device)
#train(model1, train_dataset, num_epoch)

model2 = DeepLabV3Plus(num_classes=2).to(device)
load_trained_model(model2, 100)

#train(model2, train_dataset, num_epoch, 1e-4)

#model3 = HRNetV2(num_classes=2).to(device)
#train(model3, train_dataset, num_epoch)

In [18]:
# 테스트 할 모델을 선택해주세요!
model = model2.cpu()

dataloader = DataLoader(test_dataset, batch_size=1, shuffle=True) 

for batch in dataloader:
    input, label = batch

    model.eval()

    with torch.no_grad():
        pred = torch.argmax(model(input[0].unsqueeze(dim=0)), dim=1).permute(1, 2, 0).numpy().astype(np.uint8) * 255
        label = label[0].numpy().astype(np.uint8) * 255
        input = input[0].permute(1, 2, 0).numpy().astype(np.uint8)

        pred = cv2.cvtColor(pred, cv2.COLOR_GRAY2BGR)
        label = cv2.cvtColor(label, cv2.COLOR_GRAY2BGR)

        cv2.imshow("result", cv2.hconcat((input, label, pred)))

        cv2.waitKey(0)
        cv2.destroyAllWindows()

KeyboardInterrupt: 